In [1]:
import numpy as np
import pandas as pd
import pydicom
%matplotlib inline
import matplotlib.pyplot as plt
import keras
from skimage.transform import resize
from keras.models import model_from_json

Using TensorFlow backend.


In [2]:
# This function reads in a .dcm file, checks the important fields for our device, and returns a numpy array
# of just the imaging data
def check_dicom(filename): 
    print('Load file {} ...'.format(filename))
    ds = pydicom.dcmread(filename)
    print ("Study Description {}".format(ds.StudyDescription))
    print ("Body Part Examined {}".format(ds.BodyPartExamined))
    print ("Patient Position {}".format(ds.PatientPosition))
    
    img = ds.pixel_array
    return img
    
    
# This function takes the numpy array output by check_dicom and 
# runs the appropriate pre-processing needed for our model input
def preprocess_image(img,img_mean,img_std,img_size):
    this_img_mean = np.mean(img)
    this_img_std = np.std(img)
    grey_img = (img - this_img_mean)/this_img_std
    proc_img = resize(grey_img, img_size)
    
    return proc_img

# This function loads in our trained model w/ weights and compiles it 
def load_model(model_path, weight_path):
    json_file = open(model_path, "r") 
    loaded_model_json = json_file.read() 
    json_file.close() 
    my_model = model_from_json(loaded_model_json) 
    my_model.load_weights(weight_path)
    return my_model

# This function uses our device's threshold parameters to predict whether or not
# the image shows the presence of pneumonia using our trained model
def predict_image(model, img, thresh): 
    proba = model.predict(img)
    print('Probability of pneumonia: {:.2f}'.format(proba[0][0]))
    if (proba > thresh):
        prediction = 'Pneumonia'
    else:
        prediction = 'No Pneumonia'
    
    return prediction

In [3]:
test_dicoms = ['test1.dcm','test2.dcm','test3.dcm','test4.dcm','test5.dcm','test6.dcm']
model_path = "my_model.json"
weight_path = "xray_class_my_model.best.hdf5"

IMG_SIZE=(1,224,224,3) # For vgg16
img_mean = 0 # loads the mean image value they used during training preprocessing
img_std = 0 # loads the std dev image value they used during training preprocessing

my_model = load_model(model_path, weight_path)
thresh = 0.50763667

# use the .dcm files to test prediction
for i in test_dicoms:
    img = np.array([])
    img = check_dicom(i)
    
    if img is None:
        continue
        
    img_proc = preprocess_image(img,img_mean,img_std,IMG_SIZE)
    pred = predict_image(my_model,img_proc,thresh)
    print(pred)

Load file test1.dcm ...
Study Description No Finding
Body Part Examined CHEST
Patient Position PA
Probability of pneumonia: 0.71
Pneumonia
Load file test2.dcm ...
Study Description Cardiomegaly
Body Part Examined CHEST
Patient Position AP
Probability of pneumonia: 0.72
Pneumonia
Load file test3.dcm ...
Study Description Effusion
Body Part Examined CHEST
Patient Position AP
Probability of pneumonia: 0.70
Pneumonia
Load file test4.dcm ...
Study Description No Finding
Body Part Examined RIBCAGE
Patient Position PA
Probability of pneumonia: 0.71
Pneumonia
Load file test5.dcm ...
Study Description No Finding
Body Part Examined CHEST
Patient Position PA
Probability of pneumonia: 0.71
Pneumonia
Load file test6.dcm ...
Study Description No Finding
Body Part Examined CHEST
Patient Position XX
Probability of pneumonia: 0.71
Pneumonia
